<a href="https://colab.research.google.com/github/ferrygun/ml_audio_classifier_example_for_pico/blob/main/record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa matplotlib pandas tensorflow==2.7.1 tensorflow-io==0.23.1 tensorflow-model-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.1/495.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully un

In [ ]:
import tensorflow as tf

tf.keras.utils.get_file('cmake-3.21.0-linux-x86_64.tar.gz',
                        'https://github.com/Kitware/CMake/releases/download/v3.21.0/cmake-3.21.0-linux-x86_64.tar.gz',
                        cache_dir='./',
                        cache_subdir='tools',
                        extract=True)

tf.keras.utils.get_file('gcc-arm-none-eabi-10-2020-q4-major-x86_64-linux.tar.bz2',
                        'https://developer.arm.com/-/media/Files/downloads/gnu-rm/10-2020q4/gcc-arm-none-eabi-10-2020-q4-major-x86_64-linux.tar.bz2',
                        cache_dir='./',
                        cache_subdir='tools',
                        extract=True)

156893184/156882554 [==============================] - 51s 0us/step


'./tools/gcc-arm-none-eabi-10-2020-q4-major-x86_64-linux.tar.bz2'

In [ ]:
import os

os.environ['PATH'] = f"{os.getcwd()}/tools/cmake-3.21.0-linux-x86_64/bin:{os.environ['PATH']}"
os.environ['PATH'] = f"{os.getcwd()}/tools/gcc-arm-none-eabi-10-2020-q4-major/bin:{os.environ['PATH']}"

In [ ]:
%%shell
git clone --branch 1.2.0 https://github.com/raspberrypi/pico-sdk.git
cd pico-sdk
git submodule init
git submodule update

Cloning into 'pico-sdk'...
remote: Enumerating objects: 6896, done.
remote: Counting objects: 100% (915/915), done.
remote: Compressing objects: 100% (457/457), done.
remote: Total 6896 (delta 411), reused 824 (delta 363), pack-reused 5981
Receiving objects: 100% (6896/6896), 2.73 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (3501/3501), done.
Note: switching to 'bfcbefafc5d2a210551a4d9d80b4303d4ae0adf7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

Submodule 'tinyusb' (https://github.com/hathach/tinyusb.git) registered 

In [ ]:
os.environ['PICO_SDK_PATH'] = f"{os.getcwd()}/pico-sdk"

In [ ]:
os.environ['PICO_BOARD'] = 'pico'

print(f"PICO_BOARD env. var. set to '{os.environ['PICO_BOARD']}'")

PICO_BOARD env. var. set to 'pico'


In [ ]:
%%shell
git clone https://github.com/ArmDeveloperEcosystem/microphone-library-for-pico.git

Cloning into 'microphone-library-for-pico'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 211 (delta 108), reused 195 (delta 96), pack-reused 0
Receiving objects: 100% (211/211), 51.77 KiB | 2.07 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [ ]:
%%shell
cd microphone-library-for-pico
mkdir -p build
cd build
cmake .. -DPICO_BOARD=${PICO_BOARD}

Using PICO_SDK_PATH from environment ('/content/pico-sdk')
PICO_SDK_PATH is /content/pico-sdk
Defaulting PICO_PLATFORM to rp2040 since not specified.
Defaulting PICO platform compiler to pico_arm_gcc since not specified.
-- Defaulting build type to 'Release' since not specified.
PICO compiler is pico_arm_gcc
-- The C compiler identification is GNU 10.2.1
-- The CXX compiler identification is GNU 10.2.1
-- The ASM compiler identification is GNU
-- Found assembler: /content/tools/gcc-arm-none-eabi-10-2020-q4-major/bin/arm-none-eabi-gcc
Using PICO_BOARD from environment ('pico')
Using board configuration from /content/pico-sdk/src/boards/include/boards/pico.h
-- Found Python3: /usr/bin/python3.8 (found version "3.8.10") found components: Interpreter 
TinyUSB available at /content/pico-sdk/lib/tinyusb/src/portable/raspberrypi/rp2040; adding USB support.
-- Configuring done
-- Generating done
-- Build files have been written to: /content/microphone-library-for-pico/build


In [40]:
%%shell
cd microphone-library-for-pico/build

make -j usb_microphone
#make -j hello_analog_microphone

[  2%] Performing build step for 'PioasmBuild'
[  2%] Performing build step for 'ELF2UF2Build'
[  5%] Built target bs2_default
[  5%] Built target bs2_default_padded_checksummed_asm
[100%] Built target elf2uf2
[  7%] No install step for 'ELF2UF2Build'
[100%] Built target pioasm
[  7%] Completed 'ELF2UF2Build'
[  7%] No install step for 'PioasmBuild'
[  7%] Completed 'PioasmBuild'
[ 13%] Built target ELF2UF2Build
[ 18%] Built target PioasmBuild
[ 18%] Built target pico_pdm_microphone_pdm_microphone_pio_h
Consolidate compiler generated dependencies of target usb_microphone
[100%] Built target usb_microphone


In [16]:
%%shell
git clone --recurse-submodules https://github.com/ArmDeveloperEcosystem/ml-audio-classifier-example-for-pico.git

Cloning into 'ml-audio-classifier-example-for-pico'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 43.04 MiB | 24.84 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Submodule 'inference-app/lib/CMSIS_5' (https://github.com/ARM-software/CMSIS_5.git) registered for path 'inference-app/lib/CMSIS_5'
Submodule 'inference-app/lib/microphone-library-for-pico' (https://github.com/ArmDeveloperEcosystem/microphone-library-for-pico.git) registered for path 'inference-app/lib/microphone-library-for-pico'
Submodule 'inference-app/lib/pico-tflmicro' (https://github.com/raspberrypi/pico-tflmicro.git) registered for path 'inference-app/lib/pico-tflmicro'
Cloning into '/content/ml-audio-classifier-example-for-pico/inference-app/lib/CMSIS_5'...
remote: Enumerating objects: 3242, done.        
remote: Counting objects: 100% (3242/3242), done.        
remote: Compressing objects: 100% (1346/1346), done.     

In [17]:
%%shell
ln -s ml-audio-classifier-example-for-pico/colab_utils colab_utils
ln -s ml-audio-classifier-example-for-pico/inference-app inference-app

In [41]:
from colab_utils.pico import flash_pico

flash_pico('microphone-library-for-pico/build/examples/usb_microphone/usb_microphone.bin')
#flash_pico('microphone-library-for-pico/build/examples/hello_analog_microphone/hello_analog_microphone.bin')

<IPython.core.display.Javascript object>

In [42]:
from colab_utils.audio import record_wav_file

os.makedirs('datasets/custom/fire_alarm', exist_ok=True)

record_wav_file('datasets/custom/fire_alarm')

<IPython.core.display.Javascript object>

Saving file: datasets/custom/fire_alarm/19-02-2023-05-26-52.wav


In [22]:
from colab_utils.serial_monitor import run_serial_monitor

run_serial_monitor()

<IPython.core.display.Javascript object>